# 🦥 Fine-Tuning GPT-OSS-20B dengan Unsloth di Google Colab

Notebook ini akan memandu Anda untuk fine-tune model **GPT-OSS-20B** menggunakan **Unsloth** - library yang mengoptimalkan training hingga **1.5x lebih cepat** dengan **70% lebih sedikit VRAM**.

## 📋 Yang Akan Dipelajari:
1. ✅ Setup environment dan instalasi dependencies
2. ✅ Load model GPT-OSS-20B dengan 4-bit quantization
3. ✅ Konfigurasi LoRA adapters untuk parameter-efficient fine-tuning
4. ✅ Prepare dataset untuk training
5. ✅ Training dengan SFTTrainer
6. ✅ Inference dan testing model
7. ✅ Save dan export model

## 🎯 Spesifikasi:
- **Model**: OpenAI GPT-OSS-20B (21B parameters, 3.6B active)
- **Method**: QLoRA (Quantized Low-Rank Adaptation)
- **VRAM Required**: ~14GB (cocok untuk Colab T4/L4)
- **Training Speed**: 1.5x faster vs standard implementation

---

**💡 Tips**: Pastikan menggunakan GPU runtime di Colab:
- Runtime → Change runtime type → T4 GPU / L4 GPU (recommended)

Mari kita mulai! 🚀

## 1️⃣ Setup & Instalasi

Kita akan install Unsloth dan dependencies yang diperlukan. Proses ini memakan waktu ~3-5 menit.

In [ ]:
%%capture
# Install uv package manager untuk instalasi yang lebih cepat
!pip install --upgrade -qqq uv

# Install Unsloth dan dependencies
# PENTING: Gunakan versi torch>=2.8.0 dan triton>=3.4.0
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" numpy \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    torchvision bitsandbytes "transformers>=4.55.3" \
    datasets trl peft accelerate

print("✅ Instalasi selesai!")

### Verifikasi GPU Setup

Pastikan kita memiliki GPU dengan memory yang cukup (minimal 14GB untuk QLoRA).

In [ ]:
import torch

# Cek GPU yang tersedia
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 2)
    print(f"🎮 GPU: {gpu_stats.name}")
    print(f"💾 Total Memory: {max_memory} GB")
    print(f"🔧 CUDA Version: {torch.version.cuda}")
    
    if max_memory >= 14:
        print(f"\n✅ Memory cukup untuk QLoRA training!")
    else:
        print(f"\n⚠️ Memory mungkin kurang. Recommended: 14GB+ untuk GPT-OSS-20B")
else:
    print("❌ GPU tidak terdeteksi! Pastikan runtime menggunakan GPU.")

## 2️⃣ Load Model GPT-OSS-20B

Kita akan load model dengan:
- **4-bit quantization** untuk menghemat VRAM
- **Max sequence length 2048** tokens
- Model dari Unsloth yang sudah dioptimasi

In [ ]:
from unsloth import FastLanguageModel
import torch

# Konfigurasi model
max_seq_length = 2048  # Bisa dinaikkan untuk context yang lebih panjang
dtype = None  # Auto-detect: Float16 untuk Tesla T4/V100, Bfloat16 untuk Ampere+
load_in_4bit = True  # Gunakan 4-bit quantization untuk hemat VRAM

print("📥 Loading GPT-OSS-20B model...")
print("⏳ Proses ini memakan waktu 2-3 menit...\n")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b",  # Model yang sudah dioptimasi Unsloth
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    full_finetuning=False,  # Gunakan LoRA, bukan full fine-tuning
)

print("✅ Model berhasil dimuat!")
print(f"📊 Model dtype: {model.dtype}")
print(f"📏 Max sequence length: {max_seq_length}")

## 3️⃣ Konfigurasi LoRA Adapters

**LoRA (Low-Rank Adaptation)** memungkinkan kita fine-tune hanya ~1% parameters, menghemat memory dan waktu training secara signifikan.

### Parameter Explanation:
- **r=8**: LoRA rank (higher = lebih expressive, tapi lebih lambat)
- **lora_alpha=16**: Scaling factor untuk LoRA weights
- **lora_dropout=0**: Dropout rate (0 = no dropout)
- **target_modules**: Layer mana yang akan di-train

In [ ]:
# Tambahkan LoRA adapters ke model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # LoRA rank - bisa dinaikkan untuk hasil lebih baik (8, 16, 32, 64)
    target_modules=[
        "q_proj",    # Query projection
        "k_proj",    # Key projection
        "v_proj",    # Value projection
        "o_proj",    # Output projection
        "gate_proj", # Gate projection (MoE)
        "up_proj",   # Up projection (FFN)
        "down_proj", # Down projection (FFN)
    ],
    lora_alpha=16,  # Scaling factor
    lora_dropout=0,  # Dropout untuk regularization (0 = disabled)
    bias="none",  # Tidak train bias parameters
    use_gradient_checkpointing="unsloth",  # Hemat memory dengan gradient checkpointing
    random_state=3407,  # Random seed untuk reproducibility
)

print("✅ LoRA adapters berhasil ditambahkan!")
print("\n📊 Trainable parameters:")

# Hitung berapa parameter yang akan di-train
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"Trainable: {trainable_params:,} ({trainable_percent:.2f}%)")
print(f"Total: {all_params:,}")

## 4️⃣ Prepare Dataset

Kita akan menggunakan dataset publik untuk contoh. Anda bisa mengganti dengan dataset Anda sendiri.

### Format Dataset:
Dataset harus dalam format **ShareGPT/chat** dengan struktur:
```python
{
    "messages": [
        {"role": "user", "content": "Pertanyaan..."},
        {"role": "assistant", "content": "Jawaban..."}
    ]
}
```

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt

# Load dataset - contoh: multilingual reasoning dataset
# GANTI dengan dataset Anda sendiri jika perlu
print("📥 Loading dataset...")

dataset = load_dataset(
    "HuggingFaceH4/Multilingual-Thinking",  # Dataset example
    split="train[:1000]"  # Ambil 1000 samples pertama untuk demo
)

# Standardisasi format ShareGPT
dataset = standardize_sharegpt(dataset)

print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"\n📝 Contoh data:")
print(dataset[0]["messages"][:2])  # Tampilkan 2 messages pertama

### Format Dataset untuk Training

Kita perlu convert messages ke format text menggunakan chat template.

In [ ]:
# Fungsi untuk format prompts
def formatting_prompts_func(examples):
    """Convert messages ke text format menggunakan chat template"""
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        )
        for convo in convos
    ]
    return {"text": texts}

# Apply formatting
print("🔄 Formatting dataset...")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names  # Remove original columns
)

print("✅ Dataset ready untuk training!")
print(f"\n📄 Contoh formatted text (100 chars):")
print(dataset[0]["text"][:100] + "...")

### 💡 Cara Menggunakan Dataset Sendiri

Jika Anda ingin menggunakan dataset custom, gunakan kode ini:

In [ ]:
# CONTOH: Membuat dataset custom
# Uncomment dan modifikasi sesuai kebutuhan

# from datasets import Dataset

# # Data Anda dalam format list of dicts
# custom_data = [
#     {
#         "messages": [
#             {"role": "user", "content": "Apa itu Python?"},
#             {"role": "assistant", "content": "Python adalah bahasa pemrograman..."}
#         ]
#     },
#     {
#         "messages": [
#             {"role": "user", "content": "Bagaimana cara membuat function?"},
#             {"role": "assistant", "content": "Gunakan keyword 'def' untuk..."}
#         ]
#     },
#     # Tambahkan lebih banyak data...
# ]

# # Convert ke Hugging Face Dataset
# dataset = Dataset.from_list(custom_data)
# dataset = standardize_sharegpt(dataset)
# dataset = dataset.map(formatting_prompts_func, batched=True)

# print(f"✅ Custom dataset ready: {len(dataset)} samples")

## 5️⃣ Training Configuration

Setup trainer dengan hyperparameters yang optimal untuk GPT-OSS-20B.

### Hyperparameters Explanation:
- **per_device_train_batch_size**: Jumlah samples per batch (turunkan jika OOM)
- **gradient_accumulation_steps**: Akumulasi gradients untuk batch size efektif yang lebih besar
- **learning_rate**: Learning rate (2e-4 optimal untuk LoRA)
- **max_steps**: Total training steps (set -1 untuk full epoch)
- **optim**: Optimizer (adamw_8bit hemat memory)

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

# Training configuration
training_args = SFTConfig(
    # Batch settings
    per_device_train_batch_size=1,  # Batch size per GPU (turunkan jika OOM)
    gradient_accumulation_steps=4,  # Effective batch size = 1 * 4 = 4
    
    # Learning rate & scheduler
    learning_rate=2e-4,  # Learning rate untuk LoRA
    lr_scheduler_type="linear",  # Linear decay
    warmup_steps=5,  # Warmup steps
    
    # Training steps
    max_steps=60,  # Total steps (set -1 untuk full epoch)
    num_train_epochs=1,  # Jumlah epoch (jika max_steps=-1)
    
    # Optimizer
    optim="adamw_8bit",  # 8-bit optimizer untuk hemat memory
    weight_decay=0.01,  # Weight decay untuk regularization
    
    # Logging & saving
    logging_steps=1,  # Log setiap step
    output_dir="outputs",  # Output directory
    save_strategy="steps",  # Save strategy
    save_steps=20,  # Save setiap 20 steps
    
    # Precision
    fp16=not torch.cuda.is_bf16_supported(),  # FP16 jika GPU tidak support BF16
    bf16=torch.cuda.is_bf16_supported(),  # BF16 jika GPU support
    
    # Misc
    seed=3407,  # Random seed
    dataset_text_field="text",  # Field yang berisi text
    max_seq_length=max_seq_length,  # Max sequence length
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
)

print("✅ Trainer configured!")
print(f"\n📊 Training settings:")
print(f"  - Batch size: {training_args.per_device_train_batch_size}")
print(f"  - Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning rate: {training_args.learning_rate}")
print(f"  - Max steps: {training_args.max_steps}")
print(f"  - Precision: {'BF16' if training_args.bf16 else 'FP16'}")

### 🚀 Start Training!

**PERHATIAN**: Training akan memakan waktu tergantung dataset size dan max_steps.
- 60 steps dengan batch size 4: ~5-10 menit di T4/L4
- Monitor loss - seharusnya turun secara konsisten

In [ ]:
print("🚀 Starting training...\n")
print("⏰ Ini akan memakan waktu beberapa menit...")
print("📊 Monitor loss di bawah - seharusnya turun secara bertahap\n")

# Start training
trainer_stats = trainer.train()

print("\n" + "="*50)
print("🎉 TRAINING SELESAI!")
print("="*50)
print(f"\n📈 Training Statistics:")
print(f"  - Final loss: {trainer_stats.metrics['train_loss']:.4f}")
print(f"  - Total steps: {trainer_stats.global_step}")
print(f"  - Training time: {trainer_stats.metrics.get('train_runtime', 0):.2f}s")

## 6️⃣ Testing & Inference

Sekarang mari test model yang sudah di-fine-tune!

GPT-OSS mendukung **reasoning effort levels**:
- `low`: Fast inference, simple reasoning
- `medium`: Balanced (default)
- `high`: Slow but thorough reasoning

In [ ]:
from transformers import TextStreamer

# Enable inference mode untuk model
FastLanguageModel.for_inference(model)

# Fungsi helper untuk inference
def generate_response(prompt, system_message="You are a helpful assistant.", reasoning_effort="medium"):
    """
    Generate response dari model
    
    Args:
        prompt: User input/question
        system_message: System prompt
        reasoning_effort: 'low', 'medium', atau 'high'
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    
    # Apply chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort=reasoning_effort,  # Set reasoning level
    ).to(model.device)
    
    # Generate dengan streaming output
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,  # Max tokens untuk response
        temperature=0.7,  # Temperature untuk sampling (lower = lebih deterministic)
        top_p=0.9,  # Nucleus sampling
        streamer=text_streamer,  # Stream output ke console
        use_cache=True,  # Use KV cache untuk faster generation
    )
    
    return outputs

print("✅ Inference function ready!")
print("\n💡 Gunakan: generate_response('Your question here')")

### Test dengan Beberapa Contoh

Mari coba model dengan berbagai jenis pertanyaan!

In [ ]:
# Test 1: Simple question
print("="*60)
print("🧪 TEST 1: Simple Math Problem")
print("="*60)
print("\n💬 User: Solve x^2 + 5x + 6 = 0\n")
print("🤖 Assistant:\n")

generate_response(
    "Solve x^2 + 5x + 6 = 0",
    reasoning_effort="medium"
)

In [ ]:
# Test 2: Coding question
print("\n" + "="*60)
print("🧪 TEST 2: Coding Question")
print("="*60)
print("\n💬 User: Write a Python function to check if a number is prime\n")
print("🤖 Assistant:\n")

generate_response(
    "Write a Python function to check if a number is prime",
    system_message="You are an expert Python programmer.",
    reasoning_effort="low"  # Low untuk code generation yang simple
)

In [ ]:
# Test 3: Complex reasoning
print("\n" + "="*60)
print("🧪 TEST 3: Complex Reasoning")
print("="*60)
print("\n💬 User: Explain the halting problem and why it's undecidable\n")
print("🤖 Assistant:\n")

generate_response(
    "Explain the halting problem and why it's undecidable",
    reasoning_effort="high"  # High untuk reasoning yang kompleks
)

### 💭 Test dengan Pertanyaan Anda Sendiri

In [ ]:
# Ganti dengan pertanyaan Anda!
your_question = "What is the difference between AI and Machine Learning?"

print("="*60)
print("🧪 YOUR TEST")
print("="*60)
print(f"\n💬 User: {your_question}\n")
print("🤖 Assistant:\n")

generate_response(
    your_question,
    reasoning_effort="medium"
)

## 7️⃣ Save & Export Model

Ada beberapa cara untuk save model:
1. **LoRA adapters only** - Paling kecil (beberapa MB)
2. **Merged model** - Full model dengan adapters merged (16-bit)
3. **Quantized (GGUF)** - Untuk deployment (4-bit/8-bit)
4. **Push to Hugging Face Hub** - Share dengan komunitas

In [ ]:
# Option 1: Save LoRA adapters only (recommended untuk iterasi cepat)
print("💾 Saving LoRA adapters...")

lora_output_dir = "gpt-oss-20b-lora-adapters"
model.save_pretrained(lora_output_dir)
tokenizer.save_pretrained(lora_output_dir)

print(f"✅ LoRA adapters saved to: {lora_output_dir}")
print("📦 Size: ~10-50MB (hanya adapters, bukan full model)")

In [ ]:
# Option 2: Save merged model (16-bit)
# WARNING: Ini akan menggunakan banyak disk space (~40GB)

print("💾 Saving merged model (16-bit)...")
print("⚠️  Ini memerlukan ~40GB disk space\n")

merged_output_dir = "gpt-oss-20b-finetuned-merged"

model.save_pretrained_merged(
    merged_output_dir,
    tokenizer,
    save_method="merged_16bit",  # Atau "merged_4bit" untuk quantized
)

print(f"✅ Merged model saved to: {merged_output_dir}")
print("📦 Size: ~40GB (full model)")

In [ ]:
# Option 3: Export to GGUF untuk llama.cpp
# Berguna untuk deployment di CPU atau mobile devices

print("💾 Exporting to GGUF format...")

# Quantization methods: "q4_k_m", "q5_k_m", "q8_0", "f16"
quantization_method = "q4_k_m"  # 4-bit quantization (recommended)

model.save_pretrained_gguf(
    "gpt-oss-20b-finetuned",  # Folder name
    tokenizer,
    quantization_method=quantization_method,
)

print(f"✅ GGUF model exported dengan {quantization_method} quantization")
print("📦 Size: ~10-15GB (4-bit quantized)")
print("\n💡 File ini bisa digunakan dengan llama.cpp atau Ollama")

In [ ]:
# Option 4: Push to Hugging Face Hub
# Anda perlu login ke Hugging Face terlebih dahulu

from huggingface_hub import login

# Login ke Hugging Face (uncomment untuk menggunakan)
# hf_token = "hf_..."  # Ganti dengan token Anda dari https://huggingface.co/settings/tokens
# login(token=hf_token)

# # Push LoRA adapters
# repo_name = "your-username/gpt-oss-20b-finetuned"  # Ganti dengan nama repo Anda
# print(f"🚀 Pushing to Hugging Face Hub: {repo_name}")

# model.push_to_hub(
#     repo_name,
#     token=hf_token,
#     private=False,  # Set True untuk private repo
# )
# tokenizer.push_to_hub(repo_name, token=hf_token)

# print(f"✅ Model pushed to: https://huggingface.co/{repo_name}")

print("💡 Uncomment code di atas untuk push ke Hugging Face Hub")

### 🔄 Cara Load Model yang Sudah Disave

Untuk menggunakan model yang sudah di-save nanti:

In [ ]:
# Contoh: Load LoRA adapters
# from unsloth import FastLanguageModel

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="gpt-oss-20b-lora-adapters",  # Path ke saved adapters
#     max_seq_length=2048,
#     dtype=None,
#     load_in_4bit=True,
# )

# # Enable inference
# FastLanguageModel.for_inference(model)

# # Langsung bisa digunakan!
# generate_response("Your question here")

print("💡 Uncomment code di atas untuk reload saved model")

## 🎓 Kesimpulan & Next Steps

Selamat! Anda telah berhasil:
✅ Setup environment dengan Unsloth  
✅ Load GPT-OSS-20B model dengan 4-bit quantization  
✅ Configure LoRA adapters untuk efficient training  
✅ Fine-tune model dengan dataset custom  
✅ Test model dengan berbagai reasoning levels  
✅ Save dan export model dalam berbagai format  

### 📚 Sumber Belajar Lebih Lanjut:
- [Unsloth Documentation](https://docs.unsloth.ai/)
- [GPT-OSS GitHub](https://github.com/openai/gpt-oss)
- [Unsloth GitHub](https://github.com/unslothai/unsloth)

### 🚀 Next Steps:
1. **Experiment dengan hyperparameters** - Coba learning rate, LoRA rank, dll
2. **Gunakan dataset lebih besar** - Lebih banyak data = hasil lebih baik
3. **Train lebih lama** - Tingkatkan max_steps atau epochs
4. **Try different tasks** - Fine-tune untuk specific domain (coding, medical, dll)
5. **Deploy model** - Gunakan llama.cpp, vLLM, atau TGI untuk production

### ⚡ Tips untuk Hasil Optimal:
- **Dataset quality > quantity** - Fokus pada data berkualitas tinggi
- **Monitor loss curve** - Loss harus turun smooth, tidak erratic
- **Use validation set** - Split data untuk tracking overfitting
- **Experiment dengan LoRA rank** - Higher rank = lebih expressive tapi slower
- **Try different reasoning efforts** - Match dengan use case Anda

---

**🦥 Powered by Unsloth** - 1.5x faster, 70% less VRAM

**📝 Created**: 2025 | **License**: Apache 2.0

Jika ada pertanyaan, silakan cek dokumentasi atau buka issue di GitHub!

Happy fine-tuning! 🎉

## 🔧 Troubleshooting

### ❌ Out of Memory (OOM)
**Solusi:**
```python
# 1. Turunkan batch size
per_device_train_batch_size = 1

# 2. Turunkan max_seq_length
max_seq_length = 1024  # dari 2048

# 3. Gunakan gradient checkpointing
use_gradient_checkpointing = "unsloth"

# 4. Turunkan LoRA rank
r = 4  # dari 8
```

### ⚠️ Loss tidak turun
**Solusi:**
```python
# 1. Cek learning rate - mungkin terlalu kecil/besar
learning_rate = 1e-4  # Coba berbagai nilai: 1e-5, 5e-5, 1e-4, 2e-4

# 2. Tingkatkan warmup steps
warmup_steps = 10

# 3. Cek dataset - pastikan formatnya benar
print(dataset[0]['text'][:500])
```

### 🐌 Training terlalu lambat
**Solusi:**
```python
# 1. Gunakan GPU yang lebih cepat (L4 > T4)
# 2. Tingkatkan batch size jika memory cukup
per_device_train_batch_size = 2

# 3. Turunkan max_seq_length jika tidak butuh context panjang
max_seq_length = 1024
```

### 🔄 Model output tidak bagus
**Solusi:**
```python
# 1. Train lebih lama
max_steps = 200  # atau lebih

# 2. Gunakan dataset lebih besar dan berkualitas
# 3. Tingkatkan LoRA rank
r = 16  # atau 32

# 4. Coba full fine-tuning (butuh lebih banyak VRAM)
full_finetuning = True
```